In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import time

In [2]:
cwd = os.getcwd()
os.listdir(cwd)

['.ipynb_checkpoints',
 'Binary_Image_Classification_CNN_Medium_Birds.ipynb',
 'Binary_Image_Classification_CNN_Medium_Tumor-Gray_200,200.ipynb',
 'Binary_Image_Classification_CNN_Medium_Tumor_Base.ipynb',
 'Binary_Image_Classification_CNN_Medium_Tumor_Gray_224,224.ipynb',
 'Binary_Image_Classification_CNN_Medium_Tumor_Gray_512,512.ipynb',
 'Binary_Image_Classification_CNN_Medium_Tumor_RGB_512,512.ipynb',
 'Image_Classification_By_Loading_Own_Data.ipynb',
 'image_classification_sample_code.ipynb',
 'Image_Classification_Using_Keras_Birds.ipynb',
 'Image_Classification_Using_Keras_Cancer.ipynb',
 'Keras_Classification.ipynb',
 'test_bird',
 'test_tumor',
 'test_tumor1',
 'test_tumor2',
 'test_tumor3',
 'train_bird',
 'train_tumor',
 'train_tumor1',
 'train_tumor2',
 'train_tumor3',
 'validation_bird',
 'validation_tumor']

In [3]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory("train_tumor",
                                          target_size=(512,512),
                                          batch_size = 32,
                                          class_mode = 'binary')
                                         
validation_dataset = test.flow_from_directory("validation_tumor",
                                          target_size=(512,512),
                                          batch_size =32,
                                          class_mode = 'binary')

test_dataset = test.flow_from_directory("test_tumor",
                                          target_size=(512,512),
                                          batch_size =32,
                                          class_mode = 'binary')

Found 88 images belonging to 2 classes.
Found 44 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


In [4]:
validation_dataset.class_indices

{'NoTumor': 0, 'Tumor': 1}

In [5]:
train_dataset.class_indices

{'NoTumor': 0, 'Tumor': 1}

In [6]:
model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(512,512,3)))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 2
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 3
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 4
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# This layer flattens the resulting image array to 1D array
model.add(keras.layers.Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model.add(keras.layers.Dense(512,activation='relu'))

# Output layer with single neuron which gives 0 for Cat or 1 for Dog 
#Here we use sigmoid activation function which makes our model output to lie between 0 and 1
model.add(keras.layers.Dense(1,activation='sigmoid'))

In [7]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
#steps_per_epoch = train_imagesize/batch_size
start = time.time()

model.fit(train_dataset,
         epochs = 3, batch_size=10,
         validation_data = validation_dataset)

end = time.time()
print(end - start)

Epoch 1/3
3/3 [==============================] - 16s 6s/step - loss: 0.4972 - accuracy: 0.8523 - val_loss: 0.4242 - val_accuracy: 0.8636
Epoch 2/3
3/3 [==============================] - 16s 5s/step - loss: 0.5464 - accuracy: 0.8182 - val_loss: 0.2594 - val_accuracy: 0.8409
Epoch 3/3
3/3 [==============================] - 16s 5s/step - loss: 0.2440 - accuracy: 0.9659 - val_loss: 0.4548 - val_accuracy: 0.8636
49.039737701416016


In [9]:
model.evaluate(test_dataset)

1/1 [==============================] - 1s 868ms/step - loss: 0.3716 - accuracy: 0.7727


[0.3715533912181854, 0.7727272510528564]

In [10]:
"""

#steps_per_epoch = train_imagesize/batch_size

model.fit_generator(train_dataset,
         steps_per_epoch = 250,
         epochs = 10,
         validation_data = test_dataset
       
         )
         
"""

'\n\n#steps_per_epoch = train_imagesize/batch_size\n\nmodel.fit_generator(train_dataset,\n         steps_per_epoch = 250,\n         epochs = 10,\n         validation_data = test_dataset\n       \n         )\n         \n'

In [11]:
"""

# Manually Predicting 
test_path = '../test_tumor'
Tumor=0
NoTumor=0
for i in os.listdir(test_path):
    img = image.load_img(test_path + "//" +i, target_size=(512,512,3))
    plt.imshow(img)
    plt.show()
    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    images = np.vstack([X])
    val = model.predict(images)
    if val == 0:
        NoTumor +=1
        print('NoTumor')
    else:
        Tumor +=1
        print('Tumor')
print(f'Tumor: {Tumor}, NoTumor: {NoTumor}')


"""

'\n\n# Manually Predicting \ntest_path = \'../test_tumor\'\nTumor=0\nNoTumor=0\nfor i in os.listdir(test_path):\n    img = image.load_img(test_path + "//" +i, target_size=(512,512,3))\n    plt.imshow(img)\n    plt.show()\n    X = image.img_to_array(img)\n    X = np.expand_dims(X, axis=0)\n    images = np.vstack([X])\n    val = model.predict(images)\n    if val == 0:\n        NoTumor +=1\n        print(\'NoTumor\')\n    else:\n        Tumor +=1\n        print(\'Tumor\')\nprint(f\'Tumor: {Tumor}, NoTumor: {NoTumor}\')\n\n\n'

In [12]:
"""

def predictImage(filename):
    img1 = image.load_img(filename,target_size=(224,224))
    
    plt.imshow(img1)
 
    Y = image.img_to_array(img1)
    
    X = np.expand_dims(Y,axis=0)
    val = model.predict(X)
    print(val)
    if val == 1:
        
        plt.xlabel("Duck",fontsize=20)
        
    
    elif val == 0:
        
        plt.xlabel("Bird",fontsize=20)


"""

'\n\ndef predictImage(filename):\n    img1 = image.load_img(filename,target_size=(224,224))\n    \n    plt.imshow(img1)\n \n    Y = image.img_to_array(img1)\n    \n    X = np.expand_dims(Y,axis=0)\n    val = model.predict(X)\n    print(val)\n    if val == 1:\n        \n        plt.xlabel("Duck",fontsize=20)\n        \n    \n    elif val == 0:\n        \n        plt.xlabel("Bird",fontsize=20)\n\n\n'